## Simple time integration - the Euler method

In this unit we begin with Newton's second law

$$F = ma$$

and apply it to the problem of a falling body,
$$F = -mg$$

where $m$ is the mass of the body and $g$ = 9.8 $\textrm{m/s}^2$. Assume the body falls along the y-axis, then from the definition of acceleration we know

$$\frac{d^2y}{dt^2} = -g$$

Pause to note that this is called a *second-order ordinary differential equation*. Second order because it is a second derivative, ordinary because the function $y$ is a function of one variable only, $t$ and differential because it relates rates of change through derivatives. This terminology will be important!

The analytic solution to this equation can be arrived at by integrating the equation twice, yielding
$$ \frac{dy}{dt} = v(t) = v(0) -gt$$
$$y(t) = -\frac{1}{2}gt^2 + v(0) t +y(0).$$

You no doubt explored various problems that can be solved with that pair of equations in previous physics classes. They work well. For the sake of this class, and a simple starting point, let us solve these equations *numerically*. The derivatives we are interested in are:

$\frac{dy}{dt} = v$ and,
$\frac{dv}{dt} = -g$

If we recall our calculus, the definition of a derivative is the following:

$ \lim_{\Delta t \rightarrow 0} \frac{y(t+\Delta t) - y(t)}{\Delta t} = v(t)$ and,
$ \lim_{\Delta t \rightarrow 0} \frac{v(t+\Delta t) - v(t)}{\Delta t} = -g(t)$

If we assume that we can use floating point arithmetic to get a $\Delta t$ sufficiently small, then we can begin treating $\Delta t$ as an algebraic quantity, allowing us to solve the equations for what will happen next, $t+\Delta t$ in terms of now, $t$

$$y(t+\Delta t) = y(t) + v(t) \Delta t$$
$$v(t+\Delta t) = v(t) - g(t) \Delta t$$

Note that we have effectively transformed a second order ordinary differential equation, $\frac{d^2y}{dt^2} = -g$ into a pair of first order differential equations. This method amounts to using information at the beginning of the time interval to assess slopes $v(t)$ and $g(t)$. It is called the *Euler Method* and is a good starting point for studying how ODEs are solved numerically.

### Numerical solution

Now, let us write a general interface to solving such equations. It was helpful to think of Newton's second law in the context of gravity to see the relation between forces and second order ordinary differential equations. However, as we begin to write software, we'd like something more general. In very general terms, we are trying to solve equations of the form

$$\frac{d \mathbf{y}}{dt} = f(y,t)$$

where $\mathbf{y}$ is a vector with multiple values, and $f(y,t)$ returns one value for each $\mathbf{y}$. Specifically, $f(y,t)$ is a function that accepts values of $\mathbf{y}$, and returns their derivatives! If those derivatives explicitly depend on $t$, then the system is said to be *non-autonomous*. For most of our examples, the systems will be *autonomous* or independent of explicit $t$ dependence.

We'd like to abstract that $f(y,t)$ and write a function to take the system from $t$ to $\Delta t$.

### Problem 1

Write a function, called `solve_ode` according to the signature below. While it's rare that I'm so specific about the required interface to your solutions, you will later learn that if you do it this way, moving to the pre-written ODE solvers will be much easier. The challenging portions are to proper use of function handles, including the function handle for the keyword `method` and the use of `*args` and `**options` (or, often `**kwargs`). Consult Python documentation if you are not familiar with these features of the language.

Finally observe that data will have to be managed internally, and that lists are well suited to the task. However, they will have to be converted to `numpy arrays` upon return from the function.

*in-class* - discuss the following solver with the people sitting near you and come up with implementation plans. Be especially conscience of the `*args`, `**options`, function handles, and modes of storing data within a central loop over time.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def Euler(dt, f, t, y, args):
    return y + f(t,y,*args) * dt # y is the old state, f is the derivative function (rate of change), dt is the time step, args are any additional arguments to the function

def EulerCromer(dt, f, t, y, args):
    pass

def EulerRichardson():
    pass


def solve_ode(f,tspan, y0, method = Euler, *args, **options):
    """
    Given a function f that returns derivatives,
    dy / dt = f(t, y)
    and an inital state:
    y(tspan[0]) = y0
    
    This function will return the set of intermediate states of y
    from t0 (tspan[0]) to tf (tspan[1])
    
    The function is called as follows:
    
    INPUTS 
    
    f - the function handle to the function that returns derivatives of the 
        vector y at time t. The function can also accept parameters that are
        passed via *args, eg f(t,y,g) could accept the acceleration due to gravity.
        
    tspan - a indexed data type that has [t0 tf] as its two members. 
            t0 is the initial time
            tf is the final time
    
    y0 - The initial state of the system, must be passed as a numpy array.
    
    method - The method of integrating the ODEs. This week will be one of Euler, 
             Euler-Cromer, or Euler-Richardson
    
    *args - a tuple containing as many additional parameters as you would like for 
            the function handle f.
    
    **options - a dictionary containing all the keywords that might be used to control
                function behavior. For now, there is only one:
                
                first_step - the initial time step for the simulation.
    
    
    OUTPUTS
    
    t,y
    
    The returned states will be in the form of a numpy array
    t containing the times the ODEs were solved at and an array
    y with shape tsteps,N_y where tsteps is the number of steps 
    and N_y is the number of equations. Observe this makes plotting simple:
    
    plt.plot(t,y[:,0])
    
    would plot positions.
    
    """
    
    t0 = tspan[0]
    tf = tspan[1]
    y = [y0]
    t = [t0]
    
    while t[-1] < tf:
        y_new = method(dt, f, t[-1], y[-1], args) # access -1 to get the last element of the list, which is the most recent state
        y.append(y_new)
        t.append(t[-1] + dt)
        
    return np.array(t), np.array(y)

### Problem 2

Now implement the ODEs at the beginning of this discussion,

$\frac{dy}{dt} = v$ and,
$\frac{dv}{dt} = -g$

where $g$ is 9.8 $\frac{m}{s^2}$. These physics should enter as a function called `falling_body(t,y,g)`. 

Integrate this function using the Euler method. Use $\mathbf{y}_0 = (y_0,v_0) = (3,0)$. Begin at $t_0$= 0, and end at time $t_f=\sqrt{\frac{6}{g}}$. Plot your results. Label the axes with titles and appropriate units.



In [ ]:
def simple_gravity(t,y,g):
    """
    This describes the ODEs for the kinematic equations:
    dy/dt =  v
    dv/dt = -g
    """
    dydt = y[1] # access the first element of the array y, which is the velocity
    dvdt = -g
    return np.array([dydt,dvdt])

tspan = [0,10]
y0 = np.array([0,0])
g = 9.8
dt = 0.1

t,y = solve_ode(simple_gravity,tspan,y0,Euler,g)

plt.plot(t,y[:,0])

### Problem 3

*In class* - think about and see if you can think of ways of improving upon the Euler method. Be sure to discuss the core function of what Euler is doing and where it is doing it.

Now add the Euler Cromer and Euler Richardson methods for integrating the ODEs. The methods are to evaluate slopes at the beginning of the time interval, Euler, evaluating the slopes at the end of the interval, Euler Cromer, and evaluating slopes at the middle of the interval, Euler Richardson.

#### Euler Method

Slopes at the beginning:

$$y(t+\Delta t) = y(t) + \frac{dy}{dt}(t,y)\Delta t$$

#### Euler Cromer Method

Slopes at the end:

$$y_{end} = y(t) + \frac{dy}{dt}(t,y) \Delta t$$
$$y(t+\Delta t) = y(t) + \frac{dy}{dt}(t+\Delta t,y_{end}) \Delta t$$


**Note -** *a small improvement for some systems can be achieved by first finding the new postion using an Euler step,starting at time $t$ and then using the result of this to find the update to the velocity, using position $y(t+\Delta t)$. This is probably the true intent of the algorithm, but that makes the algorithm too specialized, so we'll use the formula above.*

#### Euler Richardson Method
Slopes in the middle:

$$y_{mid} = y(t) + \frac{dy}{dt}(t,y) \frac{\Delta t}{2}$$
$$y(t+\Delta t) = y(t) +  \frac{dy}{dt}(t+\frac{\Delta t}{2},y_{mid}) \Delta t$$

### Problem 4

Use the newly implemented algorithms to redo the falling body simulation. Plot all three on the same axes and create a legend. Explore how differences in accuracy are seen as $\Delta t$ is varied. What $\Delta t$ is required for good results for each of the three algorithms? For comparison and calculation of error, use the analytic solution

$$y(t) = -\frac{1}{2}gt^2+v_0t+y_0.$$


### Problem 5 

Find the slope of the curve that results when plotting error vs. $\Delta t$. Use the `loglog` function to do the plotting. Refer back to the section on fractals to learn how to do this type of plot and curve fitting. We are looking for the exponent in the relationship 

$$\textrm{error} \propto \Delta t^{\alpha}$$

Before carying out this exercise, make sure you know what a desirable $\alpha$ is vs. an undesirable $\alpha$.

Make sure to find the errors for a large range of $\Delta t$s. I suggest you become familiar with the `logspace` function in `numpy`.

In [2]:
from scipy.stats import linregress

def error_scale(steps,errors,plot=True):
    """
    INPUTS - 
        steps = a vector of the steps, this will be better if they are logspaced
        errors = a vector errors, usually |y - y_analytic|
        plot = a boolean telling the method to plot, or not
    """
    pass


def error(y,y_a):
    """
    Simple convenience to compute errors between y and y_a
    the numerical (y) and the analytic solution (y_a)
    Use a L2 norm or just a sum of the squares of differences
    """
    pass


dts = np.logspace(-1,-4,20) # You will want these spaced out over several decades


### Problem 6

Finally, implement the function describing simple harmonic motion. This is determined from the following equations

$$ \frac{d^2y}{dt^2} = -\frac{k}{m} y $$

Use $k=1$, $m = 1$, $y_0 = .5$, $t_0=0$, $t_f=2\pi \cdot 4$.

Follow the procedure used above: first plot position vs. time for each of the three algorithms, then plot the errors as a function of the time step. Are there differences in the errors? Where, how, what? The analytic solution for this system is:

$$ \omega_0 = \sqrt{\frac{k}{m}}$$

$$ y(t) = y_0 \cos(\omega_0 t) + \frac{v_0}{\omega_0} \sin(\omega_0 t) $$

In [3]:
def sho(t,y,k):
    """
    The simple harmonic oscillator
    dy/dt = v
    dv/dt = -k/m y
    """
    pass

# Plot the basic behavior of all 3 algorithms over the time of interest


In [4]:
# Conduct the complete error analysis